# Multi Query Strategy Evaluation
The simple pipeline approach can be improved by using a Multi Query strategy. The goal of this strategy is to enhance the performance of the Retrieval Augmented Generation (RAG) model, particularly the context recall metrics. The key idea is to retrieve the documents that are most relevant for answering the user's query. The hypothesis is that the user's intended question may differ from the way they have written the query.

To address this, the Multi Query strategy involves rewriting the user's query 5 times from different perspectives. This is done to uncover the relevant documents that can provide the necessary context. A chunk of text is then retrieved for each of these rewritten queries. Finally, the unique union of all these retrieved documents is used as the context for answering the original user query.

The motivation behind this approach is to better capture the user's underlying informational need, even if it is not fully reflected in the initial query formulation. By diversifying the queries and aggregating the retrieved contexts, the system aims to improve the overall performance and relevance of the responses.

In [1]:
# Importing libraries
import sys
from dotenv import load_dotenv
import pandas as pd
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/data/cnn_dailymail_3.0.0.csv'
data = file_loader.load_csv(file_path)

In [3]:
# Setting character text splitter
chunk_size= 500
chunk_overlap= 150
vectorstore_character = file_loader.character_text_splitter(data,chunk_size, chunk_overlap)

In [4]:
# Loading syntetic test data
syntetic_test_data =pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

In [5]:
# Setting retriever for character based chuncking 
retriver = vectorstore_character.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [6]:
# Adding answer to test data for character based chuncking
multiquery_answer = evaluation.adding_answer_to_testdata(syntetic_test_data,pipelines.multi_query_pipeline, vectorstore_character, retriver)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [7]:
# Evaluating character based chuncking
multiquery_rag_evaluation_result = evaluation.ragas_evaluator(multiquery_answer)

Evaluating: 100%|██████████| 80/80 [00:33<00:00,  2.36it/s]


In [10]:
# Evaluation mean for chunking size 500
result = evaluation.evaluation_mean(multiquery_rag_evaluation_result)

context_precision: 93.41%, faithfulness: 92.36%, answer_relevancy: 86.02%, context_recall: 88.5%


context_precision: 92.99%, faithfulness: 85.84%, answer_relevancy: 95.41%, context_recall: 89.5%


In [10]:
multiquery_rag_evaluation_result[['context_precision','faithfulness','answer_relevancy','context_recall']]

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.000000,0.800000,0.882616,1.000000
1,1.000000,1.000000,0.895578,1.000000
2,1.000000,0.727273,0.997624,1.000000
3,1.000000,1.000000,0.995994,0.500000
4,0.755556,1.000000,0.987858,1.000000
5,1.000000,0.750000,0.980358,1.000000
6,1.000000,1.000000,0.955961,1.000000
7,0.926667,0.666667,0.994600,0.833333
8,1.000000,1.000000,0.983625,1.000000
9,0.916667,1.000000,1.000000,1.000000
